# LAB 2

## Data preprocessing on Global Land Temperature

In [1]:
# !wget "https://raw.githubusercontent.com/dbdmg/data-science-lab/master/datasets/GLT_filtered.csv" -O GLT_filtered.csv

1. Load the Global Land Temperature dataset as a list of lists. Before starting, take a moment to better inspect the attributes you are going to work on. How many of them are nominal, how many
continuous or discrete?

In [2]:
import csv
import pandas as pd
import numpy as np

2. Analyze the attribute AverageTemperature, which contains missing values. Fill any gap with the arithmetic mean among the closest antecedent and the closest successive measurements in time, taken in the same city. Assume the following rules for edge cases:

a. it can happen that a missing value does not have a preceding (or successive) measurement. This happens when the missing value is the first (or last) value of the dataset. If this is the case, consider the missing value to be preceded (or followed) by a 0, then compute the mean accordingly

b. if there are consecutive missing values, just compute them in temporal order and use the newly inserted values to evaluate the following ones. Here it is an example with a simple list where both (a) and (b) rules have been applied

Definisco le funzioni per il calcolo delle medie di temperatura

In [3]:
def mean_calc_df_init(df):
    if np.isnan(df['AverageTemperature_filled']) and df['row_num'] == 1:
        return df['next_not_null_AvgTemp']/2
    else: 
        return df['AverageTemperature_filled']

def mean_calc_df_last(df):
    if np.isnan(df['AverageTemperature_filled']):
        return df['prev_not_null_AvgTemp']/2
    else: 
        return df['AverageTemperature_filled']

def mean_calc_df(df):
    if np.isnan(df['AverageTemperature_filled']):
        v = sum([2**i for i in range(0,df['consective_nones_pos'])])
        return (df['prev_not_null_AvgTemp'] + df['next_not_null_AvgTemp']*v)/(2**(df['consective_nones_pos']))
    else: 
        return df['AverageTemperature_filled']

Importo i dati

In [4]:
GLT_filtered = pd.read_csv('GLT_filtered.csv')

Trasformo il tipo di 'Date' in date e 'AverageTemperature' in valore numerico

In [5]:
GLT_filtered['Date'] = pd.to_datetime(GLT_filtered['Date'], format = '%Y-%m-%d')
GLT_filtered['AverageTemperature'] = pd.to_numeric(GLT_filtered['AverageTemperature']) 
# pd.to_numeric(GLT_filtered['AverageTemperature'], errors='coerce')

Creo la colonna che ospiterà le temperature medie senza NAN

In [6]:
GLT_filtered['AverageTemperature_filled'] = GLT_filtered['AverageTemperature']

Aggiungo il numero del record per ciascuna sezione relativa alle città

In [7]:
GLT_filtered['row_num'] = GLT_filtered\
    .groupby(['City'])\
    .cumcount()+1

Aggiungo colonna con prime temperature medie successive non NAN

In [8]:
GLT_filtered['next_not_null_AvgTemp'] = GLT_filtered['AverageTemperature_filled']
GLT_filtered['next_not_null_AvgTemp'] = GLT_filtered\
    .groupby(['City'])['next_not_null_AvgTemp']\
    .backfill()

Do il valore iniziale alla temperatura media nel caso in cui sia NAN nel primo record relativo a una città

In [9]:
GLT_filtered['AverageTemperature_filled'] = GLT_filtered.apply(mean_calc_df_init, axis=1)

Aggiungo colonna con prime temperature medie precedenti non NAN

In [10]:
GLT_filtered['prev_not_null_AvgTemp'] = GLT_filtered['AverageTemperature_filled']
GLT_filtered['prev_not_null_AvgTemp'] = GLT_filtered\
    .groupby(['City'])['prev_not_null_AvgTemp']\
    .ffill()


Aggiungo colonna con check valori nulli ({0,1}) in 'AverageTemperature_filled', colonna di appoggio con cumsum e colonna di appoggio con cumsum raggruppata su colonna precedente e su 'Città' 

In [11]:
GLT_filtered['avg_is_none'] = 0
GLT_filtered.loc[GLT_filtered['AverageTemperature_filled'].isnull(), 'avg_is_none'] = 1
GLT_filtered['consective_nones'] = (GLT_filtered['avg_is_none'] != GLT_filtered['avg_is_none'].shift(1)).cumsum()
GLT_filtered['consective_nones_pos'] = GLT_filtered\
    .groupby(['City','consective_nones'])['avg_is_none']\
    .cumsum()

Applico funzione per riempire i NAN

In [12]:
GLT_filtered['AverageTemperature_filled'] = GLT_filtered.apply(mean_calc_df, axis=1)

Applico funzione per eliminare i NAN rimanenti sugli ultimi record di ogni sezione 'Città'

In [13]:
GLT_filtered['AverageTemperature_filled'] = GLT_filtered.apply(mean_calc_df_last, axis=1)

Elimino colonne di appoggio

In [14]:
GLT_filtered = GLT_filtered.drop(['row_num', 'next_not_null_AvgTemp','prev_not_null_AvgTemp', 'avg_is_none', 'consective_nones','consective_nones_pos'], axis=1)

In [15]:
# a = GLT_filtered[GLT_filtered['AverageTemperature_filled'].isnull()]
# b = GLT_filtered.groupby('City').first()
# c = GLT_filtered.groupby('City').last()

2. Define a function that, given the name of a city and an integer N > 0, prints:  
    a. the top N hottest measurements;  
    b. the top N coldest measurements

In [16]:
def top_temperatures_city(df, name):
    df_out = df[df['City'] == name] 
    df_out_1 = df_out.nlargest(10,'AverageTemperature_filled')
    df_out_2 = df_out.nsmallest(10,'AverageTemperature_filled')
    return (df_out_1,df_out_2)
    # return(df_out)

In [17]:
highest,lowest = top_temperatures_city(df=GLT_filtered,name='Aleppo')

## Textual data preparation on IMDB reviews

1. Load the IMDb dataset as a list of lists

In [18]:
aclimdb_reviews_train = []
with open('aclimdb_reviews_train.txt', encoding="utf8") as f:
    for row in f:
        aclimdb_reviews_train.append(row)
aclimdb_reviews_train = aclimdb_reviews_train[1:]

2. Apply the tokenization function listed below to your reviews. Please refer to the function’s docstring 1 for the input and output parameters. The tokenization procedure splits each comment in tokens (i.e. separate words).

In [19]:
import string
def tokenize(docs):
    """Compute the tokens for each document.
    Input: a list of strings. Each item is a document to tokenize.
    Output: a list of lists. Each item is a list containing the tokens of the
    relative document.
    """
    tokens = []
    for doc in docs:
        for punct in string.punctuation:
            doc = doc.replace(punct, " ")
        split_doc = [ token.lower() for token in doc.split(" ") if token ]
        tokens.append(split_doc)
    return tokens

In [20]:
aclimdb_reviews_train_tokenized = tokenize(aclimdb_reviews_train)

# aclimdb_reviews_train_tokenized

3. The next step requires the computation of the term frequency (TF) of each token within its respective document. Although there exist different techniques to evaluate the frequency, we will now assume that the TF of a token t in a document d is equal to the number of occurrences of t in d. Compute the TF for all your reviews.

In [21]:
from collections import Counter

def words_counter(docs):
    dicts = []
    for doc in docs:
        # print(dict(Counter(doc)))
        dicts.append(dict(Counter(doc)))
    return dicts

In [22]:
aclimdb_reviews_train_dicts = words_counter(aclimdb_reviews_train_tokenized)

4. We will now compute the inverse document frequency (IDF). While the TF gives an idea of the weight of a token within a document, the IDF is used to find its significance among the entire collection of documents (i.e. your reviews)

In [25]:
def set_words_all(docs):
    set_words = set()
    for doc in docs:
        set_words = set_words.union(set(doc))
    return set_words

def set_words_doc(docs):
    set_words_list = list()
    for doc in docs:
        set_words_list = set_words_list.append(set(doc))
    return set_words_list

In [24]:
aclimdb_reviews_train_set_words = set_words_all(aclimdb_reviews_train_tokenized)

In [27]:
# aclimdb_reviews_train_list_of_set_words = set_words_doc(aclimdb_reviews_train_tokenized)
set_words_all([['a','a','a','a','a','1','1','1','1','v']])

{'1', 'a', 'v'}